# 🧠 TEMPLATE — FEATURE ENGINEERING NOTEBOOK

## Objetivo:

Criar, transformar e selecionar features que melhorem a performance do modelo,
a generalização e a interpretabilidade, com base nos insights da EDA.

Princípios:
- Features devem ter justificativa analítica
- Evitar leakage
- Ser reproduzível
- Facilitar deploy

## 📚 1. Imports e Configurações

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    MinMaxScaler
)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from typing import List
from src.config import DATA_PATH_PROCESSED, NUMERIC_COLS, CATEGORICAL_COLS, TARGET_COL, ID_COL

pd.set_option("display.max_columns", None)


In [ ]:
df = pd.read_csv(DATA_PATH_PROCESSED)

df.head()

## 🧹 2. Separação de Features e Target

In [ ]:
X = df.drop(columns=[TARGET_COL, ID_COL], errors="ignore")
y = df[TARGET_COL]

## 🔍 3. Validações de Segurança (Leakage Check)

### 📌 Checklist

-Feature contém informação futura? <br/>
-Feature é derivada do target? <br/>
-Feature só existe após o evento? <br/>

## 🔧 4. Feature Engineering — Numéricas

### 4.1 Tratamento de Missing

In [ ]:
from sklearn.impute import SimpleImputer

numeric_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

### 📌 Checklist

-Missing é informativo? <br/>
-Criar flag de missing? <br/>

### 4.2 Transformações

**Possíveis transformações:**
- Log
- Binning
- Normalização
- Padronização

In [ ]:
df["log_feature"] = np.log1p(df["feature_exemplo"])

### 4.3 Binning

In [ ]:
df["tenure_bucket"] = pd.cut(
    df["tenure"],
    bins=[0, 12, 24, 48, 60, np.inf],
    labels=["0-12", "12-24", "24-48", "48-60", "60+"]
)


## 🧩 5. Feature Engineering — Categóricas

### 5.1 Redução de Cardinalidade

In [ ]:
def group_rare_categories(series, min_freq=0.05):
    freq = series.value_counts(normalize=True)
    rare = freq[freq < min_freq].index
    return series.replace(rare, "Other")


### 5.2 Encoding

In [ ]:
categorical_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

### 📌 Checklist

-Cardinalidade alta? <br/>
-Encoding adequado ao modelo? <br/>

## 🔗 6. Features Derivadas / Compostas

**Exemplos:**
- Contagem de serviços ativos
- Flags booleanas
- Razões entre variáveis

In [ ]:
df["num_active_services"] = (
    (df[["service1", "service2", "service3"]] == "Yes")
    .sum(axis=1)
)

## 🔬 7. Seleção Inicial de Features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
X_selected = selector.fit_transform(X)


### 📌 Checklist

-Features quase constantes? <br/>
-Features redundantes? <br/>

## ⚙️ 8. ColumnTransformer Final

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, NUMERIC_COLS),
        ("cat", categorical_pipeline, CATEGORICAL_COLS)
    ],
    remainder="drop"
)


## 🧪 9. Validação do Preprocessamento

In [ ]:
X_transformed = preprocessor.fit_transform(X)

X_transformed.shape

## 💾 10. Salvamento para Produção

In [ ]:
from joblib import dump

dump(preprocessor, "models/preprocessor.joblib")


## 🧠 11. Registro de Decisões

**Decisões tomadas:**
- Escalonamento: StandardScaler
- Encoding: OneHotEncoder
- Features criadas:
    - tenure_bucket
    - num_active_services

## 🚀 12. Próximos Passos

**Próximos passos:**
- Modelagem
- Cross-validation
- Hyperparameter tuning
- Monitoramento de drift